In [100]:
# Load relevant packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as sm
import warnings

import re 
def clean_text(string,replace=''):
    regex = '[^A-Za-z0-9]+'
    return re.sub(regex, replace, string)

In [101]:
## load data set
carmax_df = pd.read_csv('CaseDataRemodeled.csv')

## Add Car Class (e.g. SUV)




In [102]:
import requests
from bs4 import BeautifulSoup as bsoup
    
carClass = {}
bodyStyle = ['convertible','coupe','van','truck','suv','sedan','wagon']

for style in bodyStyle:
    #assign variable for url from which we acquire html data
    search_url = 'https://www.cars.com/research/{0}/2015/?pageNum=0&rpp=120'.format(style)
                
    #use requests pack to pull dat. Headers makes it think browser and not programatic
    response = requests.get(search_url, headers={
            "User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36"}).content
            
    """ parse html """
    parsed_html = bsoup(response, 'lxml')
    """ initialize results list """
    my_result_list = []

    #for loop to dig into the table and create a list of lists
    for result in parsed_html.find_all('h4', class_="cui-delta"):
        results1 = result.text.split(' ', 2)
        my_result_list.append(results1[1:])

    carClass[style] = my_result_list

In [103]:
for i in carClass:
    for j in carClass[i]:
        for k in range(len(j)):
            j[k] = j[k].upper()

Mercedes-Benz has class and number <br>
Rename to just class

In [104]:
#Clean car models in carClass dict
for key, value in carClass.items():
    for val in value:
        val[1] = clean_text(re.sub('-CLASS','',val[1]))

In [98]:
#carmax_clean.loc[carmax_clean[carmax_clean['purchase_make'] == 'MERCEDES-BENZ'], 'purchase_model'] #= value
carmax_clean.loc[carmax_clean.purchase_make == 'MERCEDES-BENZ', 'purchase_model'] = ''.join(filter(str.isalpha, carmax_clean.loc[carmax_clean.purchase_make == 'MERCEDES-BENZ', 'purchase_model'])).split('AMG')
#carmax_clean.purchase_make

In [150]:
#Clean car models in carClass dict
carmax_clean = carmax_df.copy()
carmax_clean['purchase_model'] = carmax_clean['purchase_model'].apply(clean_text)

def clean_MB(string,replace=''):
    string = re.sub('AMG', replace, string)
    regex = '[^A-Z]+'
    return re.sub(regex, replace, string)

carmax_clean.loc[carmax_clean.purchase_make == 'MERCEDES-BENZ', 'purchase_model'] = carmax_clean.loc[carmax_clean.purchase_make == 'MERCEDES-BENZ', 'purchase_model'].apply(clean_MB)

In [151]:
np.unique(carmax_clean.loc[carmax_clean.purchase_make == 'MERCEDES-BENZ', 'purchase_model'])

array(['C', 'CL', 'CLK', 'CLKBS', 'CLS', 'E', 'G', 'GL', 'GLK', 'ML', 'R',
       'S', 'SL', 'SLK', 'SPRINTER'], dtype=object)

In [152]:
carmax_clean

,insert_num,purchase_make,purchase_model,purchase_vehicle_year,purchase_price_LB,purchase_price_UB,trade_in,vehicle_financing,customer_age,customer_income,customer_gender,customer_previous_purchase,customer_distance_to_dealer,post_purchase_satisfaction,vehicle_warranty_used,subsequent_purchases,AgeBin
0,81690,DODGE,CHARGER,2012,15001.0,20000.0,1,1,21 - 30,40001 - 60000,U,0,8.0,?,0,1,Twenties
1,109994,FORD,F150,2007,15001.0,20000.0,0,0,51 - 60,0 - 20000,F,1,19.0,?,0,0,Fifties
2,11399,BMW,328,2010,25001.0,30000.0,1,1,41 - 50,60001 - 80000,F,1,21.0,?,0,0,Forties
3,214156,LEXUS,GS300,2003,10001.0,15000.0,0,1,21 - 30,20001 - 40000,M,0,8.0,?,0,0,Twenties
4,36685,CHEVROLET,CRUZE,2012,15001.0,20000.0,1,1,31 - 40,120001 - 140000,M,1,5.0,?,1,1,Thirties
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355869,195314,JEEP,LIBERTY,2010,15001.0,20000.0,0,1,21 - 30,20001 - 40000,F,0,13.0,?,0,0,Twenties
355870,58219,CHEVROLET,SONIC,2012,10001.0,15000.0,1,0,71 - 80,?,U,0,2.0,?,0,0,Seventies
355871,207386,KIA,SORENTO,2012,15001.0,20000.0,1,1,61 - 70,20001 - 40000,F,1,9.0,?,0,1,Sixties
355872,252391,NISSAN,350Z,2003,10001.0,15000.0,0,1,21 - 30,40001 - 60000,M,0,5.0,?,0,0,Twenties


In [154]:
# for i in carClass:
#     for j in carClass[i]:
#         if j[0] == 'MERCEDES-BENZ':
#             print(j[1])

In [157]:
purchase_class = []
for i in range(len(list(carmax_clean['purchase_model']))):
    carType = ''
    for key, value in carClass.items():
        for style in value:
            if carmax_clean['purchase_model'][i] == style[1]:
                carType = key
                break
            
    if carType == '':
        carType = 'NF'
    purchase_class.append(carType)
purchase_class

['sedan',
 'truck',
 'wagon',
 'NF',
 'sedan',
 'sedan',
 'truck',
 'suv',
 'suv',
 'sedan',
 'sedan',
 'sedan',
 'suv',
 'suv',
 'wagon',
 'NF',
 'NF',
 'truck',
 'sedan',
 'wagon',
 'suv',
 'NF',
 'sedan',
 'NF',
 'truck',
 'sedan',
 'sedan',
 'NF',
 'NF',
 'NF',
 'sedan',
 'suv',
 'sedan',
 'truck',
 'van',
 'sedan',
 'NF',
 'NF',
 'NF',
 'sedan',
 'NF',
 'NF',
 'sedan',
 'suv',
 'NF',
 'sedan',
 'sedan',
 'wagon',
 'suv',
 'NF',
 'NF',
 'van',
 'NF',
 'suv',
 'sedan',
 'sedan',
 'sedan',
 'sedan',
 'sedan',
 'coupe',
 'suv',
 'sedan',
 'sedan',
 'sedan',
 'sedan',
 'NF',
 'NF',
 'suv',
 'coupe',
 'NF',
 'sedan',
 'NF',
 'suv',
 'suv',
 'suv',
 'suv',
 'van',
 'truck',
 'suv',
 'suv',
 'sedan',
 'wagon',
 'NF',
 'suv',
 'NF',
 'sedan',
 'coupe',
 'sedan',
 'sedan',
 'NF',
 'wagon',
 'NF',
 'suv',
 'wagon',
 'suv',
 'sedan',
 'NF',
 'sedan',
 'NF',
 'suv',
 'truck',
 'wagon',
 'wagon',
 'sedan',
 'NF',
 'coupe',
 'truck',
 'sedan',
 'truck',
 'suv',
 'NF',
 'NF',
 'suv',
 'sedan',
 '

In [ ]:
from itertools import product
purchase_class = []
for i in range(len(list(carmax_clean['purchase_model']))):
    carType = ''
    for key, value in carClass.items():
        for style in value:
            if carmax_clean['purchase_model'][i] == style[1]:
                carType = key
                break
            
    if carType == '':
        carType = 'NF'
    purchase_class.append(carType)
purchase_class

In [158]:
len(purchase_class)

355874

In [160]:
carmax_df['purchase_class'] = purchase_class

In [161]:
# for i in carClass:
#     for j in carClass[i]:
#         if j[0] == 'LEXUS':
#             print(j[1])

In [178]:
carmax_df.groupby('purchase_class').count()
#df_rated.isnull().sum(axis = 0)

,insert_num,purchase_make,purchase_model,purchase_vehicle_year,purchase_price_LB,purchase_price_UB,trade_in,vehicle_financing,customer_age,customer_income,customer_gender,customer_previous_purchase,customer_distance_to_dealer,post_purchase_satisfaction,vehicle_warranty_used,subsequent_purchases,AgeBin
purchase_class,,,,,,,,,,,,,,,,,
NF,87255,87255,87255,87255,87253,87253,87255,87255,87255,87255,87255,87255,85663,87255,87255,87255,87255
convertible,2223,2223,2223,2223,2223,2223,2223,2223,2223,2223,2223,2223,2185,2223,2223,2223,2223
coupe,16390,16390,16390,16390,16389,16389,16390,16390,16390,16390,16390,16390,16093,16390,16390,16390,16390
sedan,114362,114362,114362,114362,114362,114362,114362,114362,114362,114362,114362,114362,112610,114362,114362,114362,114362
suv,77267,77267,77267,77267,77267,77267,77267,77267,77267,77267,77267,77267,75973,77267,77267,77267,77267
truck,20443,20443,20443,20443,20443,20443,20443,20443,20443,20443,20443,20443,20002,20443,20443,20443,20443
van,8529,8529,8529,8529,8529,8529,8529,8529,8529,8529,8529,8529,8365,8529,8529,8529,8529
wagon,29405,29405,29405,29405,29405,29405,29405,29405,29405,29405,29405,29405,28880,29405,29405,29405,29405


In [208]:
unlabeled = carmax_df.loc[carmax_df.purchase_class == 'NF',['purchase_make','purchase_model']].values[:,:]
unlabeled

array([['LEXUS', 'GS 300'],
       ['MINI', 'COOPER'],
       ['CHRYSLER', 'TOWN AND COUNTRY'],
       ...,
       ['FORD', 'RANGER'],
       ['JEEP', 'LIBERTY'],
       ['NISSAN', '350Z']], dtype=object)

In [215]:
np.unique(unlabeled, axis = 0)

TypeError: The axis argument to unique is not supported for dtype object

In [219]:
a = np.array([[1, 0, 0], [1, 0, 0], [2, 3, 4]])
np.unique(unlabeled    , axis=0)

TypeError: The axis argument to unique is not supported for dtype object

In [218]:
a

array([[1, 0, 0],
       [1, 0, 0],
       [2, 3, 4]])